导入库文件

In [1]:
#导入外部库
import numpy as np
from PIL import Image
from tqdm import tqdm

#导入自定义文件
from utils import *
from conv import *
from concat import *
from relu import *
# np.set_#printoptions(threshold=np.inf)
# np.set_#printoptions(precision=32, suppress=False, formatter={'float': '{:0.16}'.format})

加载FP32、INT8权重

In [2]:
from weights_int8 import qw1,Sw1,qw2,Sw2,qw3,Sw3,qw4,Sw4,qw5,Sw5
from bias_int8 import qb1,Sb1,qb2,Sb2,qb3,Sb3,qb4,Sb4,qb5,Sb5
from inout_int8 import Sconcat1,Zconcat1,Sconcat2,Zconcat2,Sconcat3,Zconcat3,Smul_out,Zmul_out,Ssub_out,Zsub_out,Sadd_out,Zadd_out
from inout_int8 import Sy1,Zy1,Sy2,Zy2,Sy3,Zy3,Sy4,Zy4,Sy5,Zy5
from inout_int8 import Sx1,Zx1,Sx2,Zx2,Sx3,Zx3,Sx4,Zx4,Sx5,Zx5

参数定义

In [3]:
img_path="./test/img/OTS_2987.jpg"#输入图片
# img_path="./test/img/OTS_2995.jpg"#输入图片

txt_dir = "./test/txt"#txt文件保存路径
txt_input_c1_uint8 = "./test/txt_hex/in/conv1_in_uint8.txt"#Conv1输入图片   1x1卷积 1*1*3*8 = 24bit
txt_input_c1 = "./test/txt_hex/in/conv1_in.txt"#Conv1输入图片   1x1卷积 1*1*3*8 = 24bit
txt_input_c2 = "./test/txt_hex/in/conv2_in.txt"#Conv2输入特征图 3x3卷积 3*3*3*8 = 216bit
txt_input_c3 = "./test/txt_hex/in/conv3_in.txt"#Conv3输入特征图 5x5卷积 5*5*6*8 = 1200bit
txt_input_c4 = "./test/txt_hex/in/conv4_in.txt"#Conv4输入特征图 7x7卷积 7*7*9*8 = 3528bit
txt_input_c5 = "./test/txt_hex/in/conv5_in.txt"#Conv5输入特征图 3x3卷积 3*3*6*8 = 432bit
txt_input_q1 = "./test/txt_hex/in/quant1_in.txt"#Concat1/2输入 conv2 --> concat1、concat2
txt_input_q2 = "./test/txt_hex/in/quant2_in.txt"#Concat2输入 conv3 --> concat2
txt_input_q3 = "./test/txt_hex/in/quant3_in.txt"#Concat3输入 conv4 --> concat3

txt_output_c1 = "./test/txt_hex/out_python/conv1_out.txt"#Conv1输出特征图
txt_output_c2 = "./test/txt_hex/out_python/conv2_out.txt"#Conv2输出特征图 
txt_output_c3 = "./test/txt_hex/out_python/conv3_out.txt"#Conv3输出特征图
txt_output_c4 = "./test/txt_hex/out_python/conv4_out.txt"#Conv4输出特征图
txt_output_c5 = "./test/txt_hex/out_python/conv5_out.txt"#Conv5输出特征图 
txt_output_q1 = "./test/txt_hex/out_python/quant1_out.txt"#Concat1/2输出 conv2 --> concat1、concat2
txt_output_q2 = "./test/txt_hex/out_python/quant2_out.txt"#Concat2输出 conv3 --> concat2
txt_output_q3 = "./test/txt_hex/out_python/quant3_out.txt"#Concat3输出 conv4 --> concat3

txt_output_c1_hex = "./test/txt_hex/out_python/conv1_out_hex.txt"#Conv1输出特征图
txt_output_c2_hex = "./test/txt_hex/out_python/conv2_out_hex.txt"#Conv2输出特征图 
txt_output_c3_hex = "./test/txt_hex/out_python/conv3_out_hex.txt"#Conv3输出特征图
txt_output_c4_hex = "./test/txt_hex/out_python/conv4_out_hex.txt"#Conv4输出特征图
txt_output_c5_hex = "./test/txt_hex/out_python/conv5_out_hex.txt"#Conv5输出特征图 
txt_output_result = "./test/txt_hex/out_python/result.txt"#后处理特征图 txt_output_result
txt_output_result_mul = "./test/txt_hex/out_python/result_mul.txt"
txt_output_result_sub = "./test/txt_hex/out_python/result_sub.txt"
#filter_shape=(output_channel,input_channel,filter_size)
#               输出通道数      输入通道数     卷积核大小
conv1_shape = (3,3,1)
conv2_shape = (3,3,3)
conv3_shape = (3,6,5)
conv4_shape = (3,6,7)
conv5_shape = (3,6,3)
#conv_value=(Kernel_size,stride,padding)
conv1_value = (1,1,0)
conv2_value = (3,1,1)
conv3_value = (5,1,2)
conv4_value = (7,1,3)
conv5_value = (3,1,1)

testbench生成函数

In [7]:
def dec2hex(dec_num,bit_length=8):
    """十进制有符号数转十六进制补码"""
    if dec_num < 0:
        dec_num += 1 << bit_length
    # 将无符号的二进制补码转换为十六进制格式
    # hex_num = hex(dec_num)[-3:-1]
    # hex_num = hex(dec_num)[2:]
    if dec_num <= 0xf:  # 忽略前缀'0x'
        hex_num = "0"+str(hex(dec_num)[2:])
    else:
        hex_num = hex(dec_num)[2:]
    return hex_num

#选取卷积输入特征图滑窗并产生输入测试向量(十六进制)
def tb_gen(img,txt_save_path,kernel_size,stride,padding):
    """
    img:输入(N,C,H,W)格式图片
    txt_save_path:testbench语句写入脚本
    kernel_size:卷积核尺寸
    stride:滑窗步长
    padding:补零边数
    """
    #padding添加
    hazy_image = np.lib.pad(img, ((0,0),(0,0),(padding,padding),(padding,padding)),"constant",constant_values=(0,0))
    #print(f"haze_image.shape:{hazy_image.shape}")#(1,3,482,642)

    #获取图片尺寸信息
    N,C,H,W = hazy_image.shape
    #print(f"N:{N},C:{C},H:{H},W:{W}")

    #得到滑窗个数计算
    num_w = int((W - kernel_size + 2*padding) / stride + 1)
    num_h = int((H - kernel_size + 2*padding) / stride + 1)
    #print(f"num_w:{num_w}")
    #print(f"num_h:{num_h}")

    #滑窗选取
    with open(txt_save_path,"w+") as f:
        for i in tqdm(range(0,num_h-kernel_size+1,stride)):#H 480
            for j in range(0,num_w-kernel_size+1,stride):#W 640
                window = hazy_image[:,:,i:i + kernel_size,j :j + kernel_size]#NCHW
                #将一个滑窗的数据转化为testbench中的赋值语句
                for data_CHW in window:#取出CHW
                    data_HWC = data_CHW.transpose(1,2,0)#CHW -> HWC
                    for data_WC in data_HWC:#取出WC
                        for data_C in data_WC:#取出C
                            for data in data_C:
                                # #print(data)
                                f.write("{}".format(dec2hex(data)))
                            # f.write(",")
                f.write("\n")

    print("testbench生成完毕！")
    
#产生卷积输出结果
def result_gen(img,txt_save_path):
    """
    img:输入(N,C,H,W)格式图片
    txt_save_path:testbench语句写入脚本
    """
    #获取图片尺寸信息
    N,C,H,W = img.shape
    #print(f"N:{N},C:{C},H:{H},W:{W}")

    #得到滑窗个数计算
    num_w = int(W)
    num_h = int(H)
    #print(f"num_w:{num_w}")
    #print(f"num_h:{num_h}")

    #滑窗选取
    with open(txt_save_path,"w+") as f:
        for i in tqdm(range(0,num_h,1)):#H
            for j in range(0,num_w,1):#W
                window = img[:,:,i:i + 1,j :j + 1]#NCHW
                #将一个滑窗的数据转化为testbench中的赋值语句
                for data_CHW in window:#取出CHW
                    # data_CHW = data_CHW.transpose(1,2,0)#CHW -> HWC
                    for k in range(len(data_CHW)):#取出HW
                        for data_w in data_CHW[k]:#取出W
                            for data in data_w:
                                f.write("{}".format(data))
                        if k != len(data_CHW)-1:
                            f.write(",")
                        # #print(f"{k}/{len(data_CHW)}")
                f.write("\n")
    print("output生成完毕！")

#产生Quantize输入(hex=True)和输出(hex=False)的文件
def concat_gen(img,txt_save_path,hex = True):
    """
    img:输入(N,C,H,W)格式图片
    txt_save_path:testbench语句写入脚本
    """
    #获取图片尺寸信息
    N,C,H,W = img.shape
    print(f"N:{N},C:{C},H:{H},W:{W}")

    #得到滑窗个数计算
    num_w = int((W - 1) / 1 + 1)
    num_h = int((H - 1) / 1 + 1)
    print(f"num_w:{num_w}")
    print(f"num_h:{num_h}")

    #滑窗选取
    with open(txt_save_path,"w+") as f:
        for i in tqdm(range(0,num_h-1+1,1)):#H
            for j in (range(0,num_w-1+1,1)):#W
                window = img[:,:,i:i + 1,j :j + 1]#NCHW
                #将一个滑窗的数据转化为testbench中的赋值语句
                for data_CHW in window:#取出CHW
                    # data_CHW = data_CHW.transpose(1,2,0)#CHW -> HWC
                    for k in range(len(data_CHW)):#取出HW
                        for data_w in data_CHW[k]:#取出W
                            for data in data_w:
                                if hex:
                                    f.write("{}\n".format(dec2hex(data)))
                                else:
                                    f.write("{}\n".format(data))
    print("concat输入生成完毕！")

#产生卷积输出结果
def result_gen_hex(img,txt_save_path):
    """
    img:输入(N,C,H,W)格式图片
    txt_save_path:testbench语句写入脚本
    """
    #获取图片尺寸信息
    N,C,H,W = img.shape
    #print(f"N:{N},C:{C},H:{H},W:{W}")

    #得到滑窗个数计算
    num_w = int(W)
    num_h = int(H)
    #print(f"num_w:{num_w}")
    #print(f"num_h:{num_h}")

    #滑窗选取
    with open(txt_save_path,"w+") as f:
        for i in tqdm(range(0,num_h,1)):#H
            for j in range(0,num_w,1):#W
                window = img[:,:,i:i + 1,j :j + 1]#NCHW
                #将一个滑窗的数据转化为testbench中的赋值语句
                for data_CHW in window:#取出CHW
                    # data_CHW = data_CHW.transpose(1,2,0)#CHW -> HWC
                    for k in range(len(data_CHW)):#取出HW
                        for data_w in data_CHW[k]:#取出W
                            for data in data_w:
                                f.write("{}".format(dec2hex(data)))
                        # if k != len(data_CHW)-1:
                        #     # f.write(",")
                        # #print(f"{k}/{len(data_CHW)}")
                f.write("\n")
    print("output生成完毕！")

加载图像

In [8]:
hazy_input_image = Image.open("./test/img/OTS_2987.png")#图像加载
hazy_input_image = hazy_input_image.resize((640, 480))
hazy_image = (np.asarray(hazy_input_image))
hazy_image = hazy_image.transpose(2,0,1)#HWC -> CHW
hazy_image = np.expand_dims(hazy_image,axis=0)#增加一个新的维度batch_size (batch_size,C,H,W)
#print(f"haze_image.shape:{hazy_image.shape}")
#print(f"haze_image.dtype:{hazy_image.dtype}")

In [9]:
# img = Image.open(img_path)
# img = img.resize((640,480))
# img = img.convert('RGB')
# # img_path="./test/img/OTS_2987.jpg"#输入图片
# img.save( "./test/img/OTS_2987.png", 'PNG')

输入图片量化

In [10]:
haze_img_int8 = (hazy_image-128).astype(np.int8)
print(f"haze_img_int8:{haze_img_int8}")

haze_img_int8:[[[[ 13  13  13 ...  26  26  26]
   [ 13  13  13 ...  26  26  26]
   [ 13  13  13 ...  26  26  26]
   ...
   [ -2  -1   0 ...   0 -13 -21]
   [ -2   0   1 ...   1 -12 -20]
   [ -2   0   1 ...   2 -11 -19]]

  [[ 57  57  57 ...  63  63  63]
   [ 57  57  57 ...  63  63  63]
   [ 57  57  57 ...  63  63  63]
   ...
   [ 31  32  33 ...   5  -8 -16]
   [ 31  33  34 ...   6  -7 -15]
   [ 31  33  34 ...   7  -6 -14]]

  [[ 84  84  84 ...  90  90  90]
   [ 84  84  84 ...  90  90  90]
   [ 84  84  84 ...  90  90  90]
   ...
   [ 46  47  48 ... -25 -38 -46]
   [ 46  48  49 ... -24 -37 -45]
   [ 46  48  49 ... -23 -36 -44]]]]


Conv1输入testbench

In [ ]:
# tb_gen(hazy_image,txt_input_c1_uint8,conv1_value[0],conv1_value[1],conv1_value[2])
tb_gen(haze_img_int8,txt_input_c1,conv1_value[0],conv1_value[1],conv1_value[2])

100%|██████████| 960/960 [00:10<00:00, 93.66it/s]

testbench生成完毕！


conv-relu1计算

量化卷积计算int8*int8+int32 -> int32

In [11]:
q_conv1 = Q_conv2d_fused(haze_img_int8,qw1,qb1,Sx1,Zx1,Sw1,Sb1,Sy1,Zy1,conv1_shape,stride=1,padding='SAME',const_num=Zx1)
print(f"q_conv1:{q_conv1}")
print(f"q_conv1.shape:{q_conv1.shape}")
print(f"q_conv1.dtype:{q_conv1.dtype}")

q_conv1:[[[[-146 -146 -146 ... -144 -144 -144]
   [-146 -146 -146 ... -144 -144 -144]
   [-146 -146 -146 ... -144 -144 -144]
   ...
   [-109 -110 -110 ...  -17  -10   -6]
   [-109 -110 -111 ...  -17  -11   -7]
   [-109 -110 -111 ...  -18  -11   -7]]

  [[-168 -168 -168 ... -164 -164 -164]
   [-168 -168 -168 ... -164 -164 -164]
   [-168 -168 -168 ... -164 -164 -164]
   ...
   [-145 -146 -146 ...  -97  -92  -89]
   [-145 -146 -146 ...  -98  -93  -90]
   [-145 -146 -146 ...  -98  -93  -90]]

  [[-120 -120 -120 ... -134 -134 -134]
   [-120 -120 -120 ... -134 -134 -134]
   [-120 -120 -120 ... -134 -134 -134]
   ...
   [-106 -107 -108 ... -128 -112 -103]
   [-106 -108 -110 ... -130 -114 -104]
   [-106 -108 -110 ... -131 -115 -105]]]]
q_conv1.shape:(1, 3, 480, 640)
q_conv1.dtype:int32


relu1计算

In [12]:
qrelu1 = Q_ReLU(q_conv1)
#print(f"qrelu1:{qrelu1}")
#print(f"qrelu1.shape:{qrelu1.shape}")
#print(f"qrelu1.dtype:{qrelu1.dtype}")

conv输出写入txt文件

In [ ]:
# result_gen_hex(qrelu1,txt_output_c1)

100%|██████████| 480/480 [00:03<00:00, 153.99it/s]

output生成完毕！


Conv2输入testbench

In [ ]:
# tb_gen(qrelu1,txt_input_c2,conv2_value[0],conv2_value[1],conv2_value[2])

100%|██████████| 480/480 [00:18<00:00, 25.77it/s]

testbench生成完毕！


conv-relu2计算

In [ ]:
#print(f"qrelu1.dtype:{qrelu1.dtype}")#q_x2
#print(f"qw2.dtype:{qw2.dtype}")

In [16]:
q_conv2 = Q_conv2d_fused(qrelu1,qw2,qb2,Sx2,Zx2,Sw2,Sb2,Sy2,Zy2,conv2_shape,stride=1,padding='SAME',const_num=Zx2)
#print(f"q_conv2:{q_conv2}")
#print(f"q_conv2.shape:{q_conv2.shape}")
#print(f"q_conv2.dtype:{q_conv2.dtype}")

In [17]:
qrelu2 = Q_ReLU(q_conv2)

#print(f"qrelu2:{qrelu2}")
#print(f"qrelu2.shape:{qrelu2.shape}")
#print(f"qrelu2.dtype:{qrelu2.dtype}")

In [ ]:
# result_gen_hex(qrelu2,txt_output_c2)

100%|██████████| 480/480 [00:03<00:00, 155.85it/s]

output生成完毕！


Concat1计算

In [18]:
#concat_gen(qrelu2,txt_input_q1)
q_concat1 = requantize(qrelu2,Sy2,Zy2,Sconcat1,Zconcat1)
#concat_gen(q_concat1,txt_output_q1,hex = False)
# result_gen_hex(q_concat1,"./Buffer_test/Python_hex/requantize_conv2_out.txt")
# tb_gen(q_concat1,"./Buffer_test/Python_hex/concat1_in_2_ram.txt",conv3_value[0],conv3_value[1],conv3_value[2])

# result_gen_hex(qrelu1,"./Buffer_test/Python_hex/conv1_out.txt")
# tb_gen(qrelu1,"./Buffer_test/Python_hex/concat1_in_1_ram.txt",conv3_value[0],conv3_value[1],conv3_value[2])
#print(f"q_concat1:{q_concat1}")
q_concat1 = concat(qrelu1,q_concat1)
#print(f"q_concat1:{q_concat1}")
#print(f"q_concat1.shape:{q_concat1.shape}")
#print(f"q_concat1.dtype:{q_concat1.dtype}")

Conv3输入testbench

In [ ]:
# tb_gen(q_concat1,"./Buffer_test/Python_hex/conv3_in.txt",conv3_value[0],conv3_value[1],conv3_value[2])

conv+relu3计算

In [ ]:
#print(f"q_concat1.dtype:{q_concat1.dtype}")
#print(f"qw3.dtype:{qw3.dtype}")

In [19]:
q_conv3 = Q_conv2d_fused(q_concat1,qw3,qb3,Sx3,Zx3,Sw3,Sb3,Sy3,Zy3,conv3_shape,stride=1,padding='SAME',const_num=Zx3)
#print(f"q_conv3:{q_conv3}")
#print(f"q_conv3.shape:{q_conv3.shape}")
#print(f"q_conv3.dtype:{q_conv3.dtype}")

In [20]:
qrelu3 = Q_ReLU(q_conv3)
#print(f"qrelu3:{qrelu3}")
#print(f"qrelu3.shape:{qrelu3.shape}")
#print(f"qrelu3.dtype:{qrelu3.dtype}")

In [ ]:
# result_gen_hex(qrelu3,txt_output_c3)

concat2的计算

In [21]:
# concat_gen(qrelu3,txt_input_q2)
q_concat2_int_1 = requantize(qrelu3,Sy3,Zy3,Sconcat2,Zconcat2)
#concat_gen(q_concat2_int_1,txt_output_q2,hex = False)
# tb_gen(q_concat2_int_1,"../Buffer_test/Python_hex/concat2_in_3_ram.txt",conv4_value[0],conv4_value[1],conv4_value[2])
# result_gen_hex(q_concat2_int_1,"./Buffer_test/Python_hex/requantize_conv3_out.txt")
# tb_gen(qrelu2,"./Buffer_test/Python_hex/concat2_in_2_ram.txt",conv4_value[0],conv4_value[1],conv4_value[2])
# result_gen_hex(q_concat2_int_1,"./Buffer_test/Python_hex/requanize_conv3_out.txt")
q_concat2 = concat(qrelu2,q_concat2_int_1)
#print(f"q_concat2.shape:{q_concat2.shape}")

Conv4输入testbench

In [22]:
# tb_gen(q_concat2,txt_input_c4,conv4_value[0],conv4_value[1],conv4_value[2])

conv+relu4计算

In [23]:
#print(f"q_concat2.dtype:{q_concat2.dtype}")
#print(f"qw4.dtype:{qw4.dtype}")

In [24]:
q_conv4 = Q_conv2d_fused(q_concat2,qw4,qb4,Sx4,Zx4,Sw4,Sb4,Sy4,Zy4,conv4_shape,stride=1,padding='SAME',const_num=Zx4)
#print(f"q_conv4:{q_conv4}")
#print(f"q_conv4.shape:{q_conv4.shape}")
#print(f"q_conv4.dtype:{q_conv4.dtype}")

In [25]:
qrelu4 = Q_ReLU(q_conv4)
#print(f"qrelu4:{qrelu4}")
#print(f"qrelu4.shape:{qrelu4.shape}")
#print(f"qrelu4.dtype:{qrelu4.dtype}")

In [26]:
# result_gen_hex(qrelu4,txt_output_c4)

Concat3计算

In [27]:
#concat_gen(qrelu4,txt_input_q3)
q_concat3 = requantize(qrelu4,Sy4,Zy4,Sconcat3,Zconcat3)
# result_gen_hex(q_concat3,"./Buffer_test/Python_hex/requantize_conv4_out.txt")
# tb_gen(q_concat3,"./Buffer_test/Python_hex/concat3_in_4_ram.txt",conv5_value[0],conv5_value[1],conv5_value[2])

# tb_gen(qrelu3,"./Buffer_test/Python_hex/concat3_in_3_ram.txt",conv5_value[0],conv5_value[1],conv5_value[2])
q_concat3 = concat(qrelu3,q_concat3)
#print(f"q_concat3:{q_concat3}")
#print(f"q_concat3.shape:{q_concat3.shape}")
#print(f"q_concat3.dtype:{q_concat3.dtype}")

Conv5输入testbench

In [28]:
# tb_gen(q_concat3,txt_input_c5,conv5_value[0],conv5_value[1],conv5_value[2])

conv+relu5计算

In [29]:
#print(f"q_concat3.dtype:{q_concat3.dtype}")
#print(f"qw5.dtype:{qw5.dtype}")

In [30]:
q_conv5 = Q_conv2d_fused(q_concat3,qw5,qb5,Sx5,Zx5,Sw5,Sb5,Sy5,Zy5,conv5_shape,stride=1,padding='SAME',const_num=Zx5)
#print(f"q_conv5:{q_conv5}")
#print(f"q_conv5.shape:{q_conv5.shape}")
#print(f"q_conv5.dtype:{q_conv5.dtype}")

In [31]:
qrelu5 = Q_ReLU(q_conv5)
print(f"qrelu5:{qrelu5}")
#print(f"qrelu5.shape:{qrelu5.shape}")
#print(f"qrelu5.dtype:{qrelu5.dtype}")

qrelu5:[[[[13 27 40 ... 39 40 19]
   [27 63 67 ... 73 70 61]
   [35 63 67 ... 78 72 61]
   ...
   [34 54 64 ... 75 76 62]
   [49 62 71 ... 75 74 54]
   [13 50 50 ... 52 49 44]]

  [[ 5 32 38 ... 39 39 25]
   [23 63 69 ... 73 68 58]
   [28 62 68 ... 77 69 59]
   ...
   [29 60 59 ... 76 73 57]
   [35 59 73 ... 76 72 57]
   [26 43 56 ... 52 47 28]]

  [[ 8 36 40 ... 40 40 33]
   [29 61 70 ... 71 66 55]
   [31 62 68 ... 76 70 57]
   ...
   [35 51 65 ... 74 69 55]
   [33 63 65 ... 75 68 55]
   [20 41 52 ... 52 46 20]]]]


In [32]:
# result_gen(qrelu5,txt_output_c5)
# result_gen_hex(qrelu5,txt_output_c5)
# result_gen_hex(qrelu5,txt_output_c5_hex)
# result_gen_hex(qrelu5+128,txt_output_result_mul)

mul计算

In [33]:
q_mul = q_mul_fun(qrelu5,Sy5,Zy5,haze_img_int8,Sx1,Zx1,Smul_out,Zmul_out)
# print(f"q_mul:{q_mul}")
#print(f"q_mul.dtype:{q_mul.dtype}")
# result_gen_hex(q_mul , txt_output_result_mul)

sub计算

In [34]:
q_sub = q_sub_fun(q_mul,Smul_out,Zmul_out,qrelu5,Sy5,Zy5,Ssub_out,Zsub_out)
# print(f"q_sub:{q_sub}")
# result_gen_hex(q_sub , txt_output_result_sub)

add计算

In [35]:
q_add = q_add_fun(q_sub,Ssub_out,Zsub_out,Sadd_out,Zadd_out)
print(f"q_add:\n{q_add}")
q_add_relu = Q_ReLU(q_add)
#print(f"q_add_relu:\n{q_add_relu}")

q_add:
[[[[  8  -5 -16 ... -14 -16   3]
   [ -5 -35 -39 ... -44 -40 -32]
   [-11 -35 -39 ... -48 -43 -32]
   ...
   [-10 -25 -35 ... -44 -45 -35]
   [-24 -35 -41 ... -44 -44 -27]
   [  8 -24 -24 ... -25 -24 -17]]

  [[ 14  -9 -14 ... -14 -14  -2]
   [ -1 -35 -40 ... -44 -39 -31]
   [ -5 -35 -39 ... -46 -40 -31]
   ...
   [ -6 -32 -31 ... -45 -44 -29]
   [-11 -31 -44 ... -45 -43 -29]
   [ -4 -17 -29 ... -25 -21  -5]]

  [[ 11 -12 -16 ... -16 -16  -9]
   [ -6 -32 -40 ... -41 -36 -27]
   [ -9 -35 -39 ... -45 -40 -29]
   ...
   [-11 -25 -36 ... -44 -40 -27]
   [ -9 -35 -36 ... -44 -39 -27]
   [  1 -16 -25 ... -25 -20   1]]]]


In [36]:
# result_gen(q_add_relu,txt_output_result)
# result_gen(q_add,txt_output_result)
# result_gen_hex(q_add_relu,"./Buffer_test/Python_hex/result.txt")

INT8模型输出

In [37]:
def array_show_int8(array):
    deahze_img = array.astype(np.uint8)
    #print(f"deahze_img.shape:{deahze_img.shape:}")
    deahze_img = np.squeeze(deahze_img)
    #print(f"deahze_img.shape after:{deahze_img.shape:}")
    img = Image.fromarray(deahze_img.transpose(1,2,0))
    img.show()
    
q_img_out= q_add_relu+128
print(f"q_img_out:{q_img_out}")
# result_gen_hex(q_img_out,txt_output_result_sub)
# print(f"q_img_out:{q_add_relu}")
#print(f"q_img_out.shape:{q_img_out.shape}")
#print(f"q_img_out.dtype:{q_img_out.dtype}")
array_show_int8(q_img_out)

q_img_out:[[[[136 123 112 ... 114 112 131]
   [123  93  89 ...  84  88  96]
   [117  93  89 ...  80  85  96]
   ...
   [118 103  93 ...  84  83  93]
   [104  93  87 ...  84  84 101]
   [136 104 104 ... 103 104 111]]

  [[142 119 114 ... 114 114 126]
   [127  93  88 ...  84  89  97]
   [123  93  89 ...  82  88  97]
   ...
   [122  96  97 ...  83  84  99]
   [117  97  84 ...  83  85  99]
   [124 111  99 ... 103 107 123]]

  [[139 116 112 ... 112 112 119]
   [122  96  88 ...  87  92 101]
   [119  93  89 ...  83  88  99]
   ...
   [117 103  92 ...  84  88 101]
   [119  93  92 ...  84  89 101]
   [129 112 103 ... 103 108 129]]]]
